In [1]:
!pip install requests pandas


   -------- ------------------------------- 1/5 [idna]
   ---------------- ----------------------- 2/5 [charset_normalizer]
   ---------------------------------------- 5/5 [requests]



In [2]:
import requests
import pandas as pd


In [3]:
import requests

url = "https://ci.taiwan.gov.tw/dsp/Views/dataset/detail.aspx?id=water_4"
params = {
    "city": "NewTaipei"
}

response = requests.get(url, params=params)

# 檢查有沒有成功
print(response.status_code)  # 200 就是 OK

# 看到原始 JSON 字串
print(response.text)

# 直接轉成 Python 物件（dict / list）
data = response.json()
print(type(data))
print(data)


200


<!DOCTYPE html>


<html lang="zh-Hant-TW">
<head>
    <!--螢幕寬度1200px以上為電腦模式-->
    <!--螢幕寬度1200px以下為手機平版模式-->

    <!-- Global site tag (gtag.js) - Google Analytics -->
    <script async src="https://www.googletagmanager.com/gtag/js?id=G-6HTPT42ZXT"></script>
    <script>
        window.dataLayer = window.dataLayer || [];
        function gtag() { dataLayer.push(arguments); }
        gtag('js', new Date());

        gtag('config', 'G-6HTPT42ZXT');
    </script>

    <title>
	【資料集詳細資料】－民生公共物聯網-資料服務平台
</title>
    <base href="/dsp/" />

    <!--設置IE兼容模式-->
    <meta http-equiv="X-UA-Compatible" content="IE=Edge" /><meta http-equiv="Content-Type" content="text/html; charset=utf-8" />
    <link rel="icon" type="image/png" href="/dsp/favicon.png" />

    <!--SEO-->
    <meta name="description" content="提供即時資料介接與歷史資料查詢" /><meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no" /><meta name="author" content="民生公共物聯網" /><meta name="copyright" content="民生公共物聯

JSONDecodeError: Expecting value: line 3 column 1 (char 4)

In [5]:
import requests
import pandas as pd

url = "https://sta.ci.taiwan.gov.tw/STA_WaterResource_v2/v1.0/Datastreams?$expand=Thing,Observations"

response = requests.get(url)
data = response.json()

records = []

for item in data["value"]:
    # 1. 測站名稱
    station_name = item["Thing"]["properties"].get("stationName", "未知站點")

    # 2. 經緯度，有些資料在 observedArea，有些在 Thing -> Locations 裡
    if "observedArea" in item and item["observedArea"]:
        lon, lat = item["observedArea"]["coordinates"]
    elif "Thing" in item and "Locations" in item["Thing"] and len(item["Thing"]["Locations"]) > 0:
        loc = item["Thing"]["Locations"][0]
        lon, lat = loc["location"]["coordinates"]
    else:
        lon, lat = None, None

    # 3. 觀測資料
    if "Observations" in item and len(item["Observations"]) > 0:
        obs = item["Observations"][0]
        time = obs.get("phenomenonTime")
        depth = obs.get("result")
    else:
        time, depth = None, None

    records.append({
        "station_name": station_name,
        "longitude": lon,
        "latitude": lat,
        "time": time,
        "flood_depth_cm": depth
    })

df = pd.DataFrame(records)
print(df.head())

# 可選：存成 CSV
df.to_csv("flood_sensor_data.csv", index=False, encoding="utf-8-sig")


  station_name  longitude   latitude                  time  \
0     鳳仁路與澄觀路口  120.35340  22.686167  2025-11-10T13:03:34Z   
1     中福路與中光路口  120.93429  24.839981  2025-11-10T13:11:00Z   
2        柳川(7)  120.67639  24.141603  2099-12-31T00:00:00Z   
3   XZ-image01  121.76100  24.719000  2099-12-31T00:00:00Z   
4        國盛抽水站        NaN        NaN                  None   

                                      flood_depth_cm  
0                                                  0  
1                                                  0  
2  https://iapi.wra.gov.tw/v3/api/Image/0003bf84-...  
3  https://iapi.wra.gov.tw/v3/api/Image/0003fa34-...  
4                                               None  


In [ ]:
import requests
import pandas as pd

url = "https://sta.ci.taiwan.gov.tw/STA_WaterResource_v2/v1.0/Datastreams?$expand=Thing,Observations"

records = []

while url:
    print(f"Fetching: {url}")
    r = requests.get(url)
    data = r.json()

    for item in data["value"]:
        # 測站名稱
        props = item.get("Thing", {}).get("properties", {})
        station_name = props.get("stationName", "未知")

        # 經緯度：observedArea 優先，沒有就用 Locations
        lon = lat = None
        if item.get("observedArea"):
            lon, lat = item["observedArea"]["coordinates"]
        else:
            locs = item.get("Thing", {}).get("Locations", [])
            if locs:
                lon, lat = locs[0]["location"]["coordinates"]

        # 觀測資料
        obs_list = item.get("Observations", [])
        if obs_list:
            obs = obs_list[0]
            time = obs.get("phenomenonTime")
            depth = obs.get("result")
        else:
            time = depth = None

        records.append({
            "station_name": station_name,
            "longitude": lon,
            "latitude": lat,
            "time": time,
            "flood_depth_cm": depth
        })

    # 分頁：沒有 nextLink 就結束
    url = data.get("@iot.nextLink")

df = pd.DataFrame(records)
print(df.shape)
print(df.head())

# 可選：存成 CSV
df.to_csv("flood_sensor_data.csv", index=False, encoding="utf-8-sig")


Fetching: https://sta.ci.taiwan.gov.tw/STA_WaterResource_v2/v1.0/Datastreams?$expand=Thing,Observations
Fetching: https://sta.ci.taiwan.gov.tw/STA_WaterResource_v2/v1.0/Datastreams?$skip=100&$expand=Thing,Observations%28%24skip%3D0%3B%24orderby%3D%40iot.id+asc%29&$orderby=%40iot.id+asc&$skipFilter=%28%40iot.id+gt+100%29
Fetching: https://sta.ci.taiwan.gov.tw/STA_WaterResource_v2/v1.0/Datastreams?$skip=200&$expand=Thing,Observations%28%24skip%3D0%3B%24orderby%3D%40iot.id+asc%29&$orderby=%40iot.id+asc&$skipFilter=%28%40iot.id+gt+201%29
Fetching: https://sta.ci.taiwan.gov.tw/STA_WaterResource_v2/v1.0/Datastreams?$skip=300&$expand=Thing,Observations%28%24skip%3D0%3B%24orderby%3D%40iot.id+asc%29&$orderby=%40iot.id+asc&$skipFilter=%28%40iot.id+gt+301%29
Fetching: https://sta.ci.taiwan.gov.tw/STA_WaterResource_v2/v1.0/Datastreams?$skip=400&$expand=Thing,Observations%28%24skip%3D0%3B%24orderby%3D%40iot.id+asc%29&$orderby=%40iot.id+asc&$skipFilter=%28%40iot.id+gt+401%29
Fetching: https://sta.ci

In [2]:
import requests
import pandas as pd

# ✅ 只抓 description 內含「淹水深度」的 Datastream
url = (
    "https://sta.ci.taiwan.gov.tw/STA_WaterResource_v2/v1.0/"
    "Datastreams?$expand=Thing,Observations"
    "&$filter=substringof('淹水深度',description)"
)

records = []

while url:
    print(f"Fetching: {url}")
    res = requests.get(url)
    data = res.json()

    for item in data.get("value", []):
        # 單位過濾：確保是 cm（排除影像 datastream）
        uom = item.get("unitOfMeasurement", {}).get("name")
        if uom != "cm":
            continue

        # 嘗試取得觀測值
        obs = item.get("Observations", [])
        if not obs:
            continue

        result = obs[0].get("result")
        try:
            depth = float(result)
        except (TypeError, ValueError):
            continue  # 排除非數值型（例如網址）

        # 測站名稱
        station_name = item.get("Thing", {}).get("properties", {}).get("stationName", "未知")

        # 經緯度（observedArea 優先）
        lon, lat = None, None
        if item.get("observedArea"):
            lon, lat = item["observedArea"]["coordinates"]
        else:
            locs = item.get("Thing", {}).get("Locations", [])
            if locs:
                lon, lat = locs[0]["location"]["coordinates"]

        # 時間
        time = obs[0].get("phenomenonTime")

        # 加入記錄
        records.append({
            "station_name": station_name,
            "longitude": lon,
            "latitude": lat,
            "time": time,
            "flood_depth_cm": depth
        })

    # 分頁控制
    url = data.get("@iot.nextLink")

# 轉成 DataFrame
df = pd.DataFrame(records)
print(df.shape)
print(df.head())

# 存成 CSV
df.to_csv("flood_data_filtered.csv", index=False, encoding="utf-8-sig")


Fetching: https://sta.ci.taiwan.gov.tw/STA_WaterResource_v2/v1.0/Datastreams?$expand=Thing,Observations&$filter=substringof('淹水深度',description)
Fetching: https://sta.ci.taiwan.gov.tw/STA_WaterResource_v2/v1.0/Datastreams?$skip=100&$filter=substringof%28%27%E6%B7%B9%E6%B0%B4%E6%B7%B1%E5%BA%A6%27%2Cdescription%29&$expand=Thing,Observations%28%24skip%3D0%3B%24orderby%3D%40iot.id+asc%29&$orderby=%40iot.id+asc&$skipFilter=%28%40iot.id+gt+485%29
Fetching: https://sta.ci.taiwan.gov.tw/STA_WaterResource_v2/v1.0/Datastreams?$skip=200&$filter=substringof%28%27%E6%B7%B9%E6%B0%B4%E6%B7%B1%E5%BA%A6%27%2Cdescription%29&$expand=Thing,Observations%28%24skip%3D0%3B%24orderby%3D%40iot.id+asc%29&$orderby=%40iot.id+asc&$skipFilter=%28%40iot.id+gt+834%29
Fetching: https://sta.ci.taiwan.gov.tw/STA_WaterResource_v2/v1.0/Datastreams?$skip=300&$filter=substringof%28%27%E6%B7%B9%E6%B0%B4%E6%B7%B1%E5%BA%A6%27%2Cdescription%29&$expand=Thing,Observations%28%24skip%3D0%3B%24orderby%3D%40iot.id+asc%29&$orderby=%40io

In [3]:
import requests
import pandas as pd

# 回到最原始的 Datastreams（不要先 filter，免得砍太多）
url = (
    "https://sta.ci.taiwan.gov.tw/STA_WaterResource_v2/v1.0/"
    "Datastreams?$expand=Thing,Observations"
)

records = []

while url:
    print(f"Fetching: {url}")
    res = requests.get(url)
    data = res.json()

    for item in data.get("value", []):
        # 1) 觀測資料：沒有 Observations 就略過
        obs_list = item.get("Observations", [])
        if not obs_list:
            continue

        obs = obs_list[0]
        result = obs.get("result")

        # 2) 嘗試把 result 轉成數字，失敗就跳過（多半是網址或亂碼）
        try:
            depth = float(result)
        except (TypeError, ValueError):
            continue

        # 3) 把明顯是假時間（例如 2099-12-31 這種 placeholder）踢掉
        time = obs.get("phenomenonTime")
        if time is None or time >= "2050-01-01":
            continue

        # 4) 測站名稱
        props = item.get("Thing", {}).get("properties", {})
        station_name = props.get("stationName", "未知")

        # 5) 經緯度：observedArea 優先，沒有就看 Locations
        lon = lat = None
        if item.get("observedArea"):
            lon, lat = item["observedArea"]["coordinates"]
        else:
            locs = item.get("Thing", {}).get("Locations", [])
            if locs:
                lon, lat = locs[0]["location"]["coordinates"]

        # 6) 加入紀錄
        records.append({
            "station_name": station_name,
            "longitude": lon,
            "latitude": lat,
            "time": time,
            "flood_depth_cm": depth
        })

    # 7) 分頁
    url = data.get("@iot.nextLink")

df = pd.DataFrame(records)

print("總筆數:", df.shape[0])
print(df.head())
df.to_csv("flood_data.csv", index=False, encoding="utf-8-sig")


Fetching: https://sta.ci.taiwan.gov.tw/STA_WaterResource_v2/v1.0/Datastreams?$expand=Thing,Observations
Fetching: https://sta.ci.taiwan.gov.tw/STA_WaterResource_v2/v1.0/Datastreams?$skip=100&$expand=Thing,Observations%28%24skip%3D0%3B%24orderby%3D%40iot.id+asc%29&$orderby=%40iot.id+asc&$skipFilter=%28%40iot.id+gt+100%29
Fetching: https://sta.ci.taiwan.gov.tw/STA_WaterResource_v2/v1.0/Datastreams?$skip=200&$expand=Thing,Observations%28%24skip%3D0%3B%24orderby%3D%40iot.id+asc%29&$orderby=%40iot.id+asc&$skipFilter=%28%40iot.id+gt+201%29
Fetching: https://sta.ci.taiwan.gov.tw/STA_WaterResource_v2/v1.0/Datastreams?$skip=300&$expand=Thing,Observations%28%24skip%3D0%3B%24orderby%3D%40iot.id+asc%29&$orderby=%40iot.id+asc&$skipFilter=%28%40iot.id+gt+301%29
Fetching: https://sta.ci.taiwan.gov.tw/STA_WaterResource_v2/v1.0/Datastreams?$skip=400&$expand=Thing,Observations%28%24skip%3D0%3B%24orderby%3D%40iot.id+asc%29&$orderby=%40iot.id+asc&$skipFilter=%28%40iot.id+gt+401%29
Fetching: https://sta.ci

In [1]:
import requests
import pandas as pd

url = "https://sta.ci.taiwan.gov.tw/STA_WaterResource_v2/v1.0/Datastreams?$expand=Thing,Observations"
records = []

while url:
    print("Fetching:", url)
    r = requests.get(url)
    data = r.json()

    for item in data.get("value", []):
        thing = item.get("Thing", {})
        props = thing.get("properties", {})
        authority = props.get("authority", "未知")
        station_name = props.get("stationName", "未知")

        obs = item.get("Observations", [])
        if not obs:
            continue

        result = obs[0].get("result")
        try:
            depth = float(result)
        except (TypeError, ValueError):
            continue

        time = obs[0].get("phenomenonTime")
        lon = lat = None
        if item.get("observedArea"):
            lon, lat = item["observedArea"]["coordinates"]

        records.append({
            "authority": authority,        # 哪個水利局或縣市
            "station_name": station_name,  # 測站名
            "longitude": lon,
            "latitude": lat,
            "time": time,
            "flood_depth_cm": depth
        })

    url = data.get("@iot.nextLink")

df = pd.DataFrame(records)
df.to_csv("flood_data_with_authority.csv", index=False, encoding="utf-8-sig")


Fetching: https://sta.ci.taiwan.gov.tw/STA_WaterResource_v2/v1.0/Datastreams?$expand=Thing,Observations
Fetching: https://sta.ci.taiwan.gov.tw/STA_WaterResource_v2/v1.0/Datastreams?$skip=100&$expand=Thing,Observations%28%24skip%3D0%3B%24orderby%3D%40iot.id+asc%29&$orderby=%40iot.id+asc&$skipFilter=%28%40iot.id+gt+100%29
Fetching: https://sta.ci.taiwan.gov.tw/STA_WaterResource_v2/v1.0/Datastreams?$skip=200&$expand=Thing,Observations%28%24skip%3D0%3B%24orderby%3D%40iot.id+asc%29&$orderby=%40iot.id+asc&$skipFilter=%28%40iot.id+gt+201%29
Fetching: https://sta.ci.taiwan.gov.tw/STA_WaterResource_v2/v1.0/Datastreams?$skip=300&$expand=Thing,Observations%28%24skip%3D0%3B%24orderby%3D%40iot.id+asc%29&$orderby=%40iot.id+asc&$skipFilter=%28%40iot.id+gt+301%29
Fetching: https://sta.ci.taiwan.gov.tw/STA_WaterResource_v2/v1.0/Datastreams?$skip=400&$expand=Thing,Observations%28%24skip%3D0%3B%24orderby%3D%40iot.id+asc%29&$orderby=%40iot.id+asc&$skipFilter=%28%40iot.id+gt+401%29
Fetching: https://sta.ci